# Hands-On Machine Learning with Scikit-Learn, Keras & TensorFlow
## Chapter 10 - Introduction to Artificial Neural Networks with Keras
### Imports

In [ ]:
from collections import namedtuple
import concurrent.futures
import hashlib
from pathlib import Path

import cv2
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

from tensorflow_2 import utils

In [ ]:
tf.__version__

### Variables

In [ ]:
Data = namedtuple('Data', 'features, labels')

DATA_DIR = utils.package_dir() / 'data'/ 'ch10'

---
## Load Data

In [ ]:
(x_train_full, y_train_full), (x_test, y_test) = (
    tf.keras.datasets.fashion_mnist.load_data()
)
print(f'Train Shape: {x_train_full.shape}')
print(f'Train Data Type: {x_train_full.dtype}')

### Classes

In [ ]:
classes = (
    't-shirt_top',
    'trouser',
    'pullover',
    'dress',
    'coat',
    'sandal',
    'shirt',
    'sneaker',
    'bag',
    'ankle_boot',
)
classes_idx = {n: v for n, v in enumerate(classes)}

### Create Validation Stratified Set

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    x_train_full,
    y_train_full,
    test_size=0.1,
    random_state=42,
    stratify=y_train_full
)

### Check Distributions

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=y_train, name='Train'))
fig.add_trace(go.Histogram(x=y_val, name='Validation'))
fig.add_trace(go.Histogram(x=y_test, name='Test'))

fig.update_traces(opacity=0.7)
fig.update_layout(
    title_text='Dataset Distributions',
    xaxis=dict(
        title='Class',
        tickvals=tuple(classes_idx.keys()),
        ticktext=tuple(classes_idx.values()),
    ),
    yaxis_title_text='Count',
    bargroupgap=0.1,
    barmode='group'
)

fig.show()